In [ ]:
import cv2
import mediapipe as mp
import time

In [ ]:

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()
mp_drawing = mp.solutions.drawing_utils

# Initialize OpenCV
cap = cv2.VideoCapture(0)

# Drawing settings
draw_color = (0, 255, 0)  # Green color for drawing
draw_thickness = 5
erase_thickness = 50  # Thickness of the eraser
drawing_points = []  # List to store drawing points

# Color options
colors = [(0, 255, 0), (0, 0, 255), (255, 0, 0), (255, 255, 0)]
color_index = 0

# Color selection box settings
color_box_size = 50
color_box_positions = [(10, 10), (70, 10), (130, 10), (190, 10)]

# Time tracking for clearing delay
clear_time = None
clear_delay = 5 

# Track previous points for drawing lines
prev_point = None


In [15]:

def is_middle_finger_up(thumb_tip_coords, index_tip_coords, middle_tip_coords, ring_tip_coords, pinky_tip_coords):
    # Check if middle finger is up and other fingers are down
    return (middle_tip_coords[1] < index_tip_coords[1] and
            middle_tip_coords[1] < ring_tip_coords[1] and
            middle_tip_coords[1] < thumb_tip_coords[1] and
            middle_tip_coords[1] < pinky_tip_coords[1] and
            index_tip_coords[1] > thumb_tip_coords[1] and
            ring_tip_coords[1] > thumb_tip_coords[1] and
            pinky_tip_coords[1] > thumb_tip_coords[1])


In [ ]:

# Create a named window with the ability to resize
cv2.namedWindow('Udaysinh', cv2.WINDOW_NORMAL)
# Set the desired window size
cv2.resizeWindow('Udaysinh', 1366, 768)


In [ ]:

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Flip the frame horizontally for a later selfie-view display
    frame = cv2.flip(frame, 1)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Draw color selection boxes
    for i, color in enumerate(colors):
        cv2.rectangle(frame, color_box_positions[i], (color_box_positions[i][0] + color_box_size, color_box_positions[i][1] + color_box_size), color, -1)

    # Process the frame and detect hands
    results = hands.process(frame_rgb)

    right_hand_middle_up = False
    left_hand_middle_up = False

    if results.multi_hand_landmarks:
        for hand_landmarks, handedness in zip(results.multi_hand_landmarks, results.multi_handedness):
            # hand landmarks
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # hand label (Left or Right)
            hand_label = handedness.classification[0].label

            # coordinates of thumb tip and other finger tips
            thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
            index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
            middle_tip = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP]
            ring_tip = hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_TIP]
            pinky_tip = hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_TIP]

            # Convert normalized coordinates to pixel coordinates
            h, w, _ = frame.shape
            thumb_tip_coords = (int(thumb_tip.x * w), int(thumb_tip.y * h))
            index_tip_coords = (int(index_tip.x * w), int(index_tip.y * h))
            middle_tip_coords = (int(middle_tip.x * w), int(middle_tip.y * h))
            ring_tip_coords = (int(ring_tip.x * w), int(ring_tip.y * h))
            pinky_tip_coords = (int(pinky_tip.x * w), int(pinky_tip.y * h))

            if hand_label == 'Right':
                # Check if thumb and index finger are close to each other
                distance_thumb_index = ((thumb_tip_coords[0] - index_tip_coords[0]) ** 2 + (thumb_tip_coords[1] - index_tip_coords[1]) ** 2) ** 0.5
                if distance_thumb_index < 30 and (clear_time is None or time.time() - clear_time > clear_delay):  # Threshold distance for drawing
                    if prev_point is not None:
                        drawing_points.append((prev_point, index_tip_coords, draw_color, draw_thickness))
                    prev_point = index_tip_coords
                else:
                    prev_point = None

                # Check if index finger is over a color box
                for i, (x, y) in enumerate(color_box_positions):
                    if x < index_tip_coords[0] < x + color_box_size and y < index_tip_coords[1] < y + color_box_size:
                        draw_color = colors[i]

                # Adjust thickness based on the number of fingers touching the thumb
                touching_fingers = 0
                for tip_coords in [index_tip_coords, middle_tip_coords, ring_tip_coords, pinky_tip_coords]:
                    distance = ((thumb_tip_coords[0] - tip_coords[0]) ** 2 + (thumb_tip_coords[1] - tip_coords[1]) ** 2) ** 0.5
                    if distance < 30:  # Threshold distance for touching
                        touching_fingers += 1

                draw_thickness = 5 + touching_fingers * 5  # Adjust thickness based on the number of touching fingers

                # Check if middle finger is up
                if is_middle_finger_up(thumb_tip_coords, index_tip_coords, middle_tip_coords, ring_tip_coords, pinky_tip_coords):
                    right_hand_middle_up = True

            elif hand_label == 'Left':
                # Check if hand is wide open (thumb and pinky are far apart)
                distance_thumb_pinky = ((thumb_tip_coords[0] - pinky_tip_coords[0]) ** 2 + (thumb_tip_coords[1] - pinky_tip_coords[1]) ** 2) ** 0.5
                if distance_thumb_pinky > 100:  # Threshold distance for erasing
                    # Erase points near the hand
                    margin = 10  # Margin to consider points near the border
                    drawing_points = [point for point in drawing_points if ((point[0][0] - index_tip_coords[0]) ** 2 + (point[0][1] - index_tip_coords[1]) ** 2) ** 0.5 > erase_thickness or
                                      point[0][0] < margin or point[0][0] > w - margin or point[0][1] < margin or point[0][1] > h - margin]

                # Check if middle finger is up
                if is_middle_finger_up(thumb_tip_coords, index_tip_coords, middle_tip_coords, ring_tip_coords, pinky_tip_coords):
                    left_hand_middle_up = True

    # Clear all drawings if middle finger is up on both hands
    if right_hand_middle_up and left_hand_middle_up:
        drawing_points.clear()
        clear_time = time.time()  # Record the time when the drawing is cleared

    # Draw all lines in drawing_points
    for start_point, end_point, color, thickness in drawing_points:
        cv2.line(frame, start_point, end_point, color, thickness)

    cv2.imshow('Udaysinh', frame)

    if cv2.waitKey(1) & 0xFF == 27:
        break

In [ ]:
cap.release()
cv2.destroyAllWindows()